In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd
from pathlib import Path
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from fastai.vision import *
from fastai import *

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import shutil
import torch

rn_seed=1
np.random.seed(rn_seed)
# Any results you write to the current directory are saved as output.

In [ ]:
train_path = Path('/kaggle/input/petfinder-pawpularity-score/train')
test_path = Path('/kaggle/input/petfinder-pawpularity-score/test')

train_csv = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv').dropna(axis=0)
test_csv = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

In [ ]:
for c in train_csv.columns:
    if(train_csv[c].dtype!='object'):
        train_csv[c]=train_csv[c].fillna(train_csv[c].median())

In [ ]:
train_csv.head()

In [ ]:
train_csv.describe()

In [ ]:
test_csv.head()

In [ ]:
def save_str_img(strimg,w,h,flpath):
    px=255-np.array(strimg.split(),dtype=int)
    if(len(px)==w*h and len(px)%w==0 and len(px)%h==0):
        cpx = list(px.reshape(w,h))
        img = Image(Tensor([cpx,cpx,cpx]))
        img.save(flpath)
        return img
    else:
        raise Exception("Invalid height and width")

In [ ]:
# make image folders
# shutil.rmtree(train_path)
# shutil.rmtree(test_path)

# train_path.mkdir(exist_ok=True)
# test_path.mkdir(exist_ok=True)

In [ ]:
# for each row
for index, train_row in train_csv.iterrows():
    print('train_row', train_row)

In [ ]:
def get_locs(flname):
    index = int(flname.name[:-4])
    plist=[]
    coords=list(train_csv.loc[index])
    for i in range(len(coords)//2):
        plist.append([coords[i*2+1],coords[i*2]])
    return tensor(plist)

In [ ]:
# data = (PointsItemList.from_folder(train_path)
#         .split_by_rand_pct(0.05,seed=rn_seed)
#         .label_from_func(get_locs)
#         .transform([],size=(96,96))
#         .databunch(num_workers=0)
#        )

In [ ]:
# data.show_batch(3,figsize=(6,6))

In [ ]:
def mloss(y_true, y_pred):
    y_true=y_true.view(-1,15,2)
    
    y_true[:,:,0]=y_true[:,:,0].clone()-y_pred[:,:,0]
    y_true[:,:,1]=y_true[:,:,1].clone()-y_pred[:,:,1]
    
    y_true[:,:,0]=y_true[:,:,0].clone()**2
    y_true[:,:,1]=y_true[:,:,1].clone()**2
    
    return y_true.sum(dim=2).sum(dim=1).sum()

In [ ]:
# learn = cnn_learner(train_path,models.resnet152,loss_func=mloss)